In [1]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip install argostranslate

In [2]:
import awswrangler as wr
import os
import pandas as pd

In [3]:
s3_path = "s3://accsec-ai-prod-snitchmail/processed/phish/html_files/"
html_files = wr.s3.list_objects(s3_path)
len(html_files)

2025-05-07 11:10:28,154	WARNING services.py:2022 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 1895784448 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.83gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-05-07 11:10:28,291	INFO worker.py:1786 -- Started a local Ray instance.


5560

In [ ]:
for key in html_files:
    folder = 'phish_html'
    os.makedirs(folder, exist_ok=True)
    local_file_path = os.path.join(folder, key.split('/')[-1])
    wr.s3.download(key, local_file_path)

In [37]:
phish_users_file = "s3://accsec-ai-prod-snitchmail/processed/phish/labeled_user_data.parquet"
os.makedirs('csv_files/w_label/', exist_ok=True)
wr.s3.download(phish_users_file, 'csv_files/w_label/phish_user_emails.parquet')

In [38]:
phish_user_emails_df = pd.read_parquet('csv_files/w_label/phish_user_emails.parquet')
suspended_user_data = pd.read_csv('csv_files/suspended_users_20250424.csv')

phish_user_emails_df['email_date'] = pd.to_datetime(phish_user_emails_df['email_date'])
suspended_user_data['created_at'] = pd.to_datetime(suspended_user_data['created_at'])

# Merge on user_id
phish_user_575 = pd.merge(phish_user_emails_df, suspended_user_data, on='user_id', how='inner')

# Filter: keep rows where email_date > (created_at - 3 days)
phish_user_575 = phish_user_575[phish_user_575['email_date'] > (phish_user_575['created_at'] - pd.Timedelta(days=3))]


In [39]:
phish_user_575.columns

Index(['user_id', 'msg_id', 'sg_event_id', 'subject', 'mime', 'email_from',
       'email_to', 'email_date', 'originating_ip', 'originating_ip_country',
       'lang', 's3_path', 'Content-Type', 'email_text', 'cleaned_text',
       'translated_text', 'ealvaradob_bert_finetuned_label',
       'ealvaradob_bert_finetuned_score', 'elslay_bert_finetuned_label',
       'elslay_bert_finetuned_score',
       'cybersectony_distilbert_finetuned_label',
       'cybersectony_distilbert_finetuned_score', 'created_at'],
      dtype='object')

In [40]:
languages = phish_user_emails_df['lang'].unique().tolist()
import argostranslate.package
import argostranslate.translate

to_code = "en"

for lang in languages:
    from_code = lang.split('-')[0]
    if from_code == 'en':
        continue

    # Download and install Argos Translate package
    argostranslate.package.update_package_index()
    available_packages = argostranslate.package.get_available_packages()
    # Find all matching packages
    matching_packages = [
        pkg for pkg in available_packages
        if pkg.from_code == from_code and pkg.to_code == to_code
    ]
    if not matching_packages:
        print(f"No Argos Translate package found for {from_code} to {to_code}")
        continue  # Skip to next language

    package_to_install = matching_packages[0]
    argostranslate.package.install_from_path(package_to_install.download())
    
    # Translate
    if from_code == 'zh':
        translatedText = argostranslate.translate.translate("请验证您的腾讯账户以避免使用中断", from_code, to_code)
        print(translatedText)
# '¡Hola Mundo!'

No Argos Translate package found for af to en
No Argos Translate package found for no to en
No Argos Translate package found for cy to en
No Argos Translate package found for so to en
No Argos Translate package found for hr to en
No Argos Translate package found for sw to en


/opt/conda/lib/python3.11/site-packages/stanza/models/tokenize/trainer.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, lambda storage,

Could not close temporary folder: %s
Could not close temporary folder: %s
No Argos Translate package found for vi to en


In [41]:
import argostranslate.translate

# Build a dictionary of translators for each language in your DataFrame
installed_languages = argostranslate.translate.get_installed_languages()
to_code = "en"

# Create a mapping: {from_lang_code: translator_object}
translators = {}
for lang in set(phish_user_575['lang']):
    lang = lang.split('-')[0]
    try:
        from_lang = next(l for l in installed_languages if l.code == lang)
        to_lang = next(l for l in installed_languages if l.code == to_code)
        translators[lang] = from_lang.get_translation(to_lang)
    except StopIteration:
        translators[lang] = None  # No model installed for this pair

In [42]:

def fast_translate_text(text, lang):
    lang = lang.split('-')[0]
    translator = translators.get(lang)
    if translator is not None:
        try:
            return translator.translate(text)
        except Exception:
            return text
    return text

In [43]:
# phish_user_575 = phish_user_575.drop(columns=['mime'])

In [44]:
phish_user_575['subject'] = phish_user_575.apply(lambda x: fast_translate_text(x['subject'], x['lang']), axis=1)

/opt/conda/lib/python3.11/site-packages/stanza/models/tokenize/trainer.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, lambda storage,

In [45]:
# Now use the fast function with apply
phish_user_575['translated_text'] = phish_user_575.apply(
    lambda x: fast_translate_text(x['cleaned_text'], x['lang']), axis=1
)

/opt/conda/lib/python3.11/site-packages/stanza/models/tokenize/trainer.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, lambda storage,

In [47]:
phish_user_575[phish_user_575['lang'] == 'zh-cn'].loc[5083, 'translated_text']

"(https://t.me) Using Promo Picasso to steal the competer's precision users, the social media growth program used by Top1% of the world's businesses wants to pry Facebook, Instagram, 5 billion traffic? Do you want to cut your client costs by 63% through social marketing? Multi-Account Media Matrix played this way, allowing you to support Facebook, the Instagram Multi-Account Matrix operation on the DJ 500+  * Promo Picasso: *Accessive capture of FB/IG active users, explosive billboards, hotgate owners... * Efficient customer traffic import and conversion! :: Visualize the use of security seating accounts to capture staff dynamics in real time. Click to scan login for trial (https://admin.promopicasso.com) Click to access the official channel (https://t.me) Click to consult the official passenger service (https://t.me) (https://t.me)"

In [33]:
translator = translators.get('zh')
translator.translate('Document 请立即采取行动 亲爱的 Sunjin')

'Document, please take immediate action, dear Sunjin.'

In [48]:
def create_hyperlink(file_id):
    if file_id:
        url = f"https://drive.google.com/file/d/{file_id}/view"
        return f'=HYPERLINK("{url}", "HTML File")'
    else:
        return "Not Found"
    

In [49]:
output_csv = 'csv_files/w_label/ops_labeling_575.csv'
folder_id = '17hEl7wEu-rOMSoEJLzdPICkiuYFhFlGW'

In [50]:
import json
with open('phish_html_files_map.json', 'r') as file:
    filename_to_fileid_map = json.load(file)

In [51]:
phish_user_575['filename'] = phish_user_575['msg_id'] + '.html'

In [52]:

if "filename" not in phish_user_575.columns:
    raise ValueError("Input CSV must have a 'filename' column.")

file_names = phish_user_575["filename"].tolist()

phish_user_575["HTML_File_Link"] = phish_user_575["filename"].apply(lambda name: create_hyperlink(filename_to_fileid_map.get(name)))


In [53]:
phish_user_575 = phish_user_575.drop(columns=[
    'filename', 'email_text', 'cleaned_text', 'ealvaradob_bert_finetuned_label',
       'ealvaradob_bert_finetuned_score',
       'cybersectony_distilbert_finetuned_label',
       'cybersectony_distilbert_finetuned_score',
       'elslay_bert_finetuned_label', 'elslay_bert_finetuned_score'])
phish_user_575.loc[67, 'HTML_File_Link']

'=HYPERLINK("https://drive.google.com/file/d/1meE92IXcOcxZ6rWk8pWtt0H_lw9I42Mj/view", "HTML File")'

In [60]:
phish_user_575 = phish_user_575.drop(columns=[all['Content-Type', 's3_path', 'mime'])

In [61]:
phish_user_575.to_csv(output_csv, index=False)
print(f"[SUCCESS] Updated CSV saved to: {output_csv}")

[SUCCESS] Updated CSV saved to: csv_files/w_label/ops_labeling_575.csv


In [62]:
# phish_user_emails_df['HTML_File_Link'].value_counts()
phish_user_575.columns

Index(['user_id', 'msg_id', 'sg_event_id', 'subject', 'email_from', 'email_to',
       'email_date', 'originating_ip', 'originating_ip_country', 'lang',
       'translated_text', 'created_at', 'HTML_File_Link'],
      dtype='object')

In [51]:
c = 0
for f in file_names:
    if f not in filename_to_fileid_map:
        c += 1

In [52]:
c

39

In [53]:
len(filename_to_fileid_map)

5562